In [ ]:
import numpy as np
from math import gcd
import math

MOD = 26

In [ ]:
# Utility Functions

def char_to_num(c):
    return ord(c.upper()) - ord('A')

def num_to_char(n):
    return chr((n % MOD) + ord('A'))

def text_to_numbers(text):
    return [char_to_num(c) for c in text if c.isalpha()]

def matrix_mod_inv_2x2(matrix):
    a, b = matrix[0]
    c, d = matrix[1]

    det = (a * d - b * c) % MOD

    if math.gcd(det, MOD) != 1:
        return None

    det_inv = pow(det, -1, MOD)

    inv_matrix = [
        [( d * det_inv) % MOD, (-b * det_inv) % MOD],
        [(-c * det_inv) % MOD, ( a * det_inv) % MOD]
    ]

    return inv_matrix

def matrix_mul_2x2(A, B):
    result = [[0, 0], [0, 0]]
    for i in range(2):
        for j in range(2):
            result[i][j] = (A[i][0] * B[0][j] + A[i][1] * B[1][j]) % MOD
    return result

In [ ]:
# Known Plaintext Attack

def hill_cipher_kpa(plaintext, ciphertext):
    P_nums = text_to_numbers(plaintext)
    C_nums = text_to_numbers(ciphertext)

    if len(P_nums) < 4 or len(C_nums) < 4:
        raise ValueError("Need at least 4 characters for known plaintext attack.")

    P = [
        [P_nums[0], P_nums[2]],
        [P_nums[1], P_nums[3]]
    ]

    C = [
        [C_nums[0], C_nums[2]],
        [C_nums[1], C_nums[3]]
    ]

    P_inv = matrix_mod_inv_2x2(P)
    if P_inv is None:
        return None, False

    K = matrix_mul_2x2(C, P_inv)
    return K, True

In [ ]:
# Verification

def encrypt_hill_2x2(plaintext, key):
    nums = text_to_numbers(plaintext)
    result = ""

    for i in range(0, len(nums), 2):
        if i + 1 >= len(nums):
            break

        p = [nums[i], nums[i+1]]
        c1 = (key[0][0] * p[0] + key[0][1] * p[1]) % MOD
        c2 = (key[1][0] * p[0] + key[1][1] * p[1]) % MOD

        result += num_to_char(c1)
        result += num_to_char(c2)

    return result

In [ ]:
# Main Cracker Interface

def main():
    print("\n--- Hill Cipher Known Plaintext Attack Tool ---")

    plaintext = input("Enter known plaintext (at least 4 letters): ")
    ciphertext = input("Enter corresponding ciphertext (at least 4 letters): ")

    key, success = hill_cipher_kpa(plaintext, ciphertext)

    if success:
        print("Recovered Key Matrix:")
        print(key)
        print("Verification:", encrypt_hill_2x2(plaintext, key))
    else:
        print("Attack failed")

if __name__ == "__main__":
    main()


--- Hill Cipher Known Plaintext Attack Tool ---
Enter known plaintext (at least 4 letters): TEST
Enter corresponding ciphertext (at least 4 letters): ZHKR
Recovered Key Matrix:
[[15, 0], [13, 5]]
Verification: ZHKR
